<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/Eye_ai_new_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisites
!pip install --upgrade --force pydantic git+https://github.com/fair-research/bdbag git+https://github.com/informatics-isi-edu/deriva-py git+https://github.com/informatics-isi-edu/eye-ai-tools git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
import json
import os
from eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
import torch


  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-adjk4qk6
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-adjk4qk6
  Resolved https://github.com/fair-research/bdbag to commit 4e8f87268ea4519d73eddeea7419aa1ae1ea2e37
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/informatics-isi-edu/deriva-py to /tmp/pip-req-build-218hhx5h
  Running command git clone --filter=blob:none --quiet https://github.com/informatics-isi-edu/deriva-py /tmp/pip-req-build-218hhx5h
  Resolved https://github.com/informatics-isi-edu/deriva-py to commit c1bdae58a509a70a4e46b079343eee701e9a5fe6
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/informatics-isi-edu/eye-ai-tools to /tmp/pip-req-build-c4s4m30u
  Running command git clone --filter=blob:none --quiet https://github.com/informatics-isi-edu/eye-ai-tools /tmp/pip-req-build-c4s4m30u
  Resolved https://github.com/informatics-isi-edu/eye-

In [2]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = False #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


2024-04-25 05:15:11,485 - INFO - No default configuration file found, attempting to create one at: /root/.deriva/config.json
2024-04-25 05:15:11,921 - INFO - NoSavedTokens: No tokens were loaded
Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=8ef15ba9-2b4a-469c-a163-7fd910c9d111&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Fwww.eye-ai.org%2Fderiva_all+openid+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Aview_my_groups_and_memberships&state=_default&response_type=code&code_challenge=Jb5i1ZX6UjPbDK4dYIP32gZ_om3xznbW9vdzdbCwMos&code_challenge_method=S256&access_type=online&prefill_named_grant=Login+from+deriva-client+on+5e36df61d470+%5BLinux-6.1.58%2B-x86_64-with-glibc2.35%5D+to+hosts+%5Bwww.eye-ai.org%5D+with+requested+scopes+%5Bhttps%3A%2F%2Fauth.globus.org%2Fscopes%2Fwww.eye-ai.org%2Fderiva_all%2C+openid%2C+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Av

In [3]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id, cache_dir= 'content/cache/', working_dir='content/working/')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)


In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid="2-90EP")
configuration_records.model_dump()

In [ ]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
field2_csv_path = EA.filter_angle_2(configuration_records.bag_paths[0])
field2_csv_path

In [ ]:
# @title Execute Proecss algorithm (Cropping)
from eye_ai_ml.glaucoma.optic_disk_crop import preprocess_and_crop
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  model = torch.load(model_path)

In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

In [ ]:
# @title Save cropping metadata to Image Annotation table
annot_metadata = pd.read_csv(str(EA.working_dir)+'/output.csv')
EA.insert_image_annotation(uploaded_assets[str(EA.working_dir)+'/Execution_Assets/Image_Annotation'], annot_metadata)